<a href="https://colab.research.google.com/github/sahil-ansari-15/Optimize_CNN_Model/blob/main/Optimize_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create CNN Model and Optimize it using Keras Tuner

## Data Preprocessing

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 4.0 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
# Importing the Keras libraries and packages
# import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
# from keras.layers import LeakyReLU,PReLU,ELU
# from keras.layers import Dropout

from kerastuner.tuners import RandomSearch

<ipython-input-3-dcb5267604a4>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [4]:
print(tf.__version__)

2.12.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images.shape

(60000, 28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
train_images.shape

(60000, 28, 28, 1)

## Building ANN model with ***Hyperparameters tuning***

In [11]:
def build_model(hp):  
  model = Sequential([
    Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    Flatten(),
    Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
# from kerastuner import RandomSearch
# from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [14]:
tuner.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 31s]
val_accuracy: 0.8656666874885559

Best val_accuracy So Far: 0.9129999876022339
Total elapsed time: 00h 02m 58s


In [15]:
model=tuner.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        153664    
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 112)               3469424   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 3,625,178
Trainable params: 3,625,178
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 12s 6ms/step - loss: 0.1148 - accuracy: 0.9581 - val_loss: 0.2585 - val_accuracy: 0.9155
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0788 - accuracy: 0.9706 - val_loss: 0.3248 - val_accuracy: 0.9140
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0543 - accuracy: 0.9804 - val_loss: 0.3579 - val_accuracy: 0.9150
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0387 - accuracy: 0.9858 - val_loss: 0.4393 - val_accuracy: 0.9133
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0324 - accuracy: 0.9886 - val_loss: 0.4380 - val_accuracy: 0.9147
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0253 - accuracy: 0.9912 - val_loss: 0.4730 - val_accuracy: 0.9197
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0236 - accuracy: 0.9919 - val_loss: 0.5129 - val_a